In [39]:
import os 
import pandas as pd 

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import GradientBoostingClassifier
from imblearn.over_sampling import BorderlineSMOTE
import dask.dataframe as dd

try:
    os.chdir("C:/Users/Sam/Documents/SISE/Fouille de données")
except:
    os.chdir("/Users/titouanhoude/Documents/GitHub")

data = pd.read_csv("guillaume.txt", sep = ";")

/var/folders/46/7rlt2f9x4szchbk_4z7ycdtc0000gn/T/ipykernel_1020/2060927226.py:21: DtypeWarning: Columns (1,2,5,6,7,8,9,15,16,22) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("guillaume.txt", sep = ";")


In [40]:
data.info()

data.head()

data.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4646774 entries, 0 to 4646773
Data columns (total 23 columns):
 #   Column                    Dtype 
---  ------                    ----- 
 0   ZIBZIN                    object
 1   IDAvisAutorisationCheque  object
 2   FlagImpaye                object
 3   Montant                   object
 4   DateTransaction           object
 5   CodeDecision              object
 6   VerifianceCPT1            object
 7   VerifianceCPT2            object
 8   VerifianceCPT3            object
 9   D2CB                      object
 10  ScoringFP1                object
 11  ScoringFP2                object
 12  ScoringFP3                object
 13  TauxImpNb_RB              object
 14  TauxImpNB_CPM             object
 15  EcartNumCheq              object
 16  NbrMagasin3J              object
 17  DiffDateTr1               object
 18  DiffDateTr2               object
 19  DiffDateTr3               object
 20  CA3TRetMtt                object
 21  CA3TR   

ZIBZIN                      0
IDAvisAutorisationCheque    0
FlagImpaye                  0
Montant                     0
DateTransaction             0
CodeDecision                0
VerifianceCPT1              0
VerifianceCPT2              0
VerifianceCPT3              0
D2CB                        0
ScoringFP1                  0
ScoringFP2                  0
ScoringFP3                  0
TauxImpNb_RB                0
TauxImpNB_CPM               0
EcartNumCheq                0
NbrMagasin3J                0
DiffDateTr1                 0
DiffDateTr2                 0
DiffDateTr3                 0
CA3TRetMtt                  0
CA3TR                       0
Heure                       0
dtype: int64

In [26]:
#recode type of variable 
var_quanti = ["Montant","VerifianceCPT1", "VerifianceCPT2", "VerifianceCPT3","D2CB","ScoringFP1","ScoringFP2","ScoringFP3","TauxImpNb_RB","TauxImpNB_CPM","EcartNumCheq","NbrMagasin3J","DiffDateTr1", "DiffDateTr2","DiffDateTr3","CA3TRetMtt","CA3TR","Heure"]

var_quali = ["FlagImpaye" , "IDAvisAutorisationCheque" , "CodeDecision"]

# split variable to have date and hour
data[['Date','Heure_split']] = data['DateTransaction'].astype(str).str.split(" " , expand=True)

for i in var_quanti :
    
    data[i] = data[i].replace(",", ".", regex= True)#.astype(float).round(0)

for i in var_quali :
    
    data[i] = data[i].astype(object)


In [28]:
data.FlagImpaye.value_counts()

,ZIBZIN,IDAvisAutorisationCheque,Montant,DateTransaction,CodeDecision,VerifianceCPT1,VerifianceCPT2,VerifianceCPT3,D2CB,ScoringFP1,...,EcartNumCheq,NbrMagasin3J,DiffDateTr1,DiffDateTr2,DiffDateTr3,CA3TRetMtt,CA3TR,Heure,Date,Heure_split
FlagImpaye,,,,,,,,,,,,,,,,,,,,,
0,1273110,3775285,42403,3072525,5,13,21,26,551,2063468,...,15235,5,153631,50465,12205,66860,34580,49463,302,49463
1,15152,23053,11822,23034,4,10,18,25,551,17672,...,216,4,4325,1410,463,13646,4101,17832,302,17832
0,31475,32627,11975,25605,4,5,9,10,551,26968,...,220,4,4392,747,149,13626,3528,21933,3,21933
1,134,140,130,140,4,6,8,8,93,101,...,14,2,34,10,7,132,33,140,3,140
FlagImpaye,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,0


In [23]:
for i in var_quanti :

    #data[i] = data[i].apply(pd.to_numeric, downcast = "float", errors = "coerce")
    data[i] = data[i].replace(",", ".", regex= True)#.astype(float).round(0)

#data[var_quanti] = data[var_quanti].apply(pd.to_numeric, downcast = "float", errors = "coerce")

for i in var_quali :
    
    data[i] = data[i].astype(object)
    
#look the number modality 
data.FlagImpaye.value_counts()

0             4584151
0               32627
1               29855
1                 140
FlagImpaye          1
Name: FlagImpaye, dtype: int64

In [ ]:
index = 0
for i in data['Date'] : 
    
    
    if i =="2017-09-01":
        break
    else:
        index += 1
        train = data.iloc[:index, :]


test = data.iloc[index:, :]


train.to_csv("C:/Users/Sam/Documents/GitHub/FouilleDonneeMassive/train.csv")
test.to_csv("C:/Users/Sam/Documents/GitHub/FouilleDonneeMassive/test.csv")

In [10]:
import os 
import pandas as pd 
import numpy as np

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import GradientBoostingClassifier
from imblearn.over_sampling import BorderlineSMOTE
import dask.dataframe as dd

try:
    os.chdir("C:/Users/Sam/Documents/SISE/Fouille de données")
except:
    os.chdir("/Users/titouanhoude/Documents/GitHub")
    
train = pd.read_parquet('train.parquet.gzip')
test = pd.read_parquet('test.parquet.gzip')

train.info()
test.info()

Xtrain = train.drop(["FlagImpaye", "ZIBZIN", "Date", "Heure_split", "DateTransaction", "Unnamed: 0"], axis = 1)

Ytrain = pd.DataFrame(train.FlagImpaye)
Ytrain = Ytrain['FlagImpaye'].astype('int')

Xtest  = test.drop(["FlagImpaye","ZIBZIN", "Date", "Heure_split", "DateTransaction", 'Unnamed: 0'], axis = 1)
Ytest  = test.FlagImpaye

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pandas/io/parsers/c_parser_wrapper.py:230: RuntimeWarning: overflow encountered in cast
  chunks = self._reader.read_low_memory(nrows)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pandas/io/parsers/c_parser_wrapper.py:230: RuntimeWarning: overflow encountered in cast
  chunks = self._reader.read_low_memory(nrows)


In [11]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

Xtrain_selected = SelectKBest(chi2, k=5).fit_transform(Xtrain, Ytrain)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


ValueError: Input X contains infinity or a value too large for dtype('float16').

In [43]:
from joblib import Parallel, delayed
import joblib

names=[]
f1score_ =[]

models={'SVC': SVC(),
       'RandomForest': RandomForestClassifier(),
       'DecisionTree': DecisionTreeClassifier(),
       'Naïve Bayes': GaussianNB(), 
       'Neural Network': MLPClassifier(),
       'knn' : KNeighborsClassifier(),
       'LDA': LinearDiscriminantAnalysis(),
       'GradientBoosting' : GradientBoostingClassifier(), 
       }
   
def test():
    
     sm = BorderlineSMOTE(random_state = 0)
     XBdSmote , YBdSmote = sm.fit_resample(Xtrain, Ytrain)
        
     for name, model in models.items(): 
          
        name_model = model
        name_fit = name_model.fit(XBdSmote , YBdSmote)
        name_pred = name_fit.predict(Xtest)
        f1score = f1_score(Ytest,name_pred, average = "macro")
        names.append(name)
        f1score_.append(f1score)
    
        
     score_df = pd.DataFrame(list(zip(names, f1score_)))
     score_df.columns = ["Nom", "Score"]
    
     return score_df 

results = Parallel(n_jobs=3)(delayed(test)() for _ in range(1))

results.to_csv("res.csv")